Source: https://docs.opencv.org/master/d7/d53/tutorial_py_pose.html
## Goal
In this section,
* We will learn to exploit calib3d module to create some 3D effects in images.

In [3]:
import numpy as np
import cv2 as cv
import glob

In [4]:
npz_file = np.load("c_calib.npz")
npz_file.files

['arr_0', 'arr_1', 'arr_2', 'arr_3']

In [7]:
# Saved as: np.savez(file, mtx, dist, rvecs, tvecs)
mtx, dist, _, _ = [npz_file[i] for i in npz_file.files]

Now let's create a function, draw which takes the corners in the chessboard (obtained using [cv.findChessboardCorners()](https://docs.opencv.org/master/d9/d0c/group__calib3d.html#ga93efa9b0aa890de240ca32b11253dd4a)) and axis points to draw a 3D axis.

In [19]:
def draw(img, corners, imgpts):
    # Mert: Added .astype("uint8")
    corner = tuple(corners[0].ravel().astype("uint8"))
    img = cv.line(img, corner, tuple(imgpts[0].ravel().astype("uint8")), (255,0,0), 5)
    img = cv.line(img, corner, tuple(imgpts[1].ravel().astype("uint8")), (0,255,0), 5)
    img = cv.line(img, corner, tuple(imgpts[2].ravel().astype("uint8")), (0,0,255), 5)
    return img

In [46]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
axis = np.float32([
    [3,0,0],
    [0,-3,0],
    [0,0,-3]
]).reshape(-1,3)

In [47]:
for fname in glob.glob('./sample_calibration_images/calibration_pose*.png'):
    img = cv.imread(fname)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(gray, (7,6),None)
    if ret == True:
        corners2 = cv.cornerSubPix(gray, corners,(11,11), (-1,-1), criteria)
        # Find the rotation and translation vectors.
        ret, rvecs, tvecs = cv.solvePnP(objp, corners2, mtx, dist)
        # project 3D points to image plane
        imgpts, jac = cv.projectPoints(axis, rvecs, tvecs, mtx, dist)
        img = draw(img,corners2,imgpts)
        cv.imshow('img',img)
        k = cv.waitKey(0) & 0xFF
        if k == ord('s'):
            cv.imwrite(fname[:6]+'.png', img)
cv.destroyAllWindows()